#### Import Necessary Python Libraries

In [1]:
# numpy & pandas
import numpy as np
import pandas as pd
from math import sqrt
from statistics import stdev 
import calendar
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import pylab as py
from scipy import stats
import seaborn as sns
import statsmodels
%matplotlib inline


#Machine learning Libraries
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### Importing the Data

In [2]:
#Importing dataset
bike = pd.read_csv("day.csv")

FileNotFoundError: [Errno 2] File day.csv does not exist: 'day.csv'

In [ ]:
#Checking the shape of the dataframe 
bike.shape

The dataframe has 730 rows and 16 columns

In [ ]:
# Checking the size of the dataframe
bike.size

#### Understanding the data

In [ ]:
#checking the dataset
bike.head(2)

In [ ]:
#checking dataset info
bike.info()

- Except one column 'dteday' which is object type, all other are either float or integer type.
- There are some fields that are categorical in nature, but are in integer/float type. Example : season, mnth, weathersit, holiday etc.
- We will analyze it further to decide whether to convert them to categorical or treat as integer

In [ ]:
#checking the statistics of data
bike.describe()

### Data quality check

In [ ]:
#checking null values
bike.isnull().sum()

In [ ]:
# visualization if there are any missing values in the dataset

import missingno as mn
mn.matrix(bike)
plt.show()

There are no missing values in the dataset

### Duplicate Checking

In [ ]:
# Creating a temp copy of original dataframe for duplicate check
temp = bike.copy()

# Checking for duplicates and dropping the entire duplicate row if any and then checking the shape whether there 
#is any difference in the shape
temp.drop_duplicates(subset=None, inplace=True)
print(temp.shape)
print(bike.shape)

Since, there is no change in the shape of the dataframe. We can conclude that there are no duplicates in the dataset

In [ ]:
# Checking if there is any relationship between casual, registered and cnt column
temp = bike[['casual','registered','cnt']]
# Creating a column whch will show the value of casual + registered
temp['total'] = temp['casual'] + temp ['registered']

In [ ]:
# Checking the correlation of the casual, registered and cnt variables 
plt.figure(figsize = (6,6))
sns.heatmap(temp.corr(), annot = True)
plt.show()

As we can clearly observe that correlation value of total(casual+registered) and cnt is 1, so we should drop 'casual' and 'registered'

In [ ]:
#dropping the unwanted columns
bike.drop(['instant','dteday','casual','registered'],axis=1,inplace=True)
bike.shape

- instant: It is an index value, so it would not contribute anything in the prediction of the dependent feature
- dteday: This has the date, Since we already have separate columns for 'year' & 'month' so we can drop it.
- casual & registered: As we already observed above that total(casual+registered) as 1 correlation with 'cnt' so we can drop both.

In [ ]:
bike.head()

#### Analyzing Season and Month

In [ ]:
sns.pairplot(bike[['season','mnth']])
plt.show()

*In order to avoid overfitting in the model, we are dropping 'mnth' from the dataset as it is showing high correlation with season*

In [ ]:
#dropping 'mnth' feature to get more interpretable results
bike.drop(["mnth"],axis=1,inplace=True)

In [ ]:
bike.head()

### Checking Skewness of dataset

#### Windspeed

In [ ]:
sns.distplot(bike['windspeed'])
print(bike['windspeed'].skew())

As we can see it has skweness of 0.67, we can transform it to normal distribution for better analysis

#### Humidity

In [ ]:
sns.distplot(bike['hum'])
print(bike['hum'].skew())

As we can see it has skweness of -0.067, we can transform it to normal distribution for better analysis

#### Temp

In [ ]:
sns.distplot(bike['temp'])
print(bike['temp'].skew())

As we can see it has skweness of -0.057, we can transform it to normal distribution for better analysis

#### atemp

In [ ]:
sns.distplot(bike['atemp'])
print(bike['atemp'].skew())

As we can see it has skweness of -0.134, we can transform it to normal distribution for better analysis

### Normalize the data using PowerTransformer

In [ ]:
#using power transformer to change the skewness of dataset
from sklearn.preprocessing import PowerTransformer
pt=PowerTransformer()

In [ ]:
windspeed_tf = pd.DataFrame(pt.fit_transform(bike[["windspeed"]]),columns=["windspeed_tf"])

In [ ]:
hum_tf = pd.DataFrame(pt.fit_transform(bike[["hum"]]),columns=["hum_tf"])

In [ ]:
temp_tf = pd.DataFrame(pt.fit_transform(bike[["temp"]]),columns=["temp_tf"])

In [ ]:
atemp_tf = pd.DataFrame(pt.fit_transform(bike[["atemp"]]),columns=["atemp_tf"])

#### Merging the dataset

In [ ]:
bike = pd.concat([bike,windspeed_tf,hum_tf,temp_tf,atemp_tf],axis=1)

In [ ]:
bike.drop(["windspeed","hum","temp","atemp"],axis=1,inplace=True)

In [ ]:
bike.head()

In [ ]:
bike.describe()

### Encoding Categorical columns
Converting season, weathersit and weekday to categorical columns

- season: converting season values as per criteria - 1: "Spring", 2 : "Summer", 3 : "Fall", 4 :"Winter" 
- weathersit: converting weathersit values as 1: "Clear",2 : "Mist", 3 : "Light_RainSnow",4 :"Heavy_RainSnow"
- weekday: converting weekday values as 0: "Sunday",1: "Monday",2 : "Tuesday", 3 : "Wednesday",4 :"Thursday",5:"Friday",6:"Saturday"

In [ ]:
bike["season"] = bike.season.map({1: "Spring", 2 : "Summer", 3 : "Fall", 4 :"Winter" })

bike["weathersit"] = bike.weathersit.map({1: "Clear",2 : "Mist", 3 : "Light_RainSnow",4 :"Heavy_RainSnow" })

bike["weekday"] = bike.weekday.map({0: "Sunday",1: "Monday",2 : "Tuesday", 3 : "Wednesday",4 :"Thursday",5:"Friday",6:"Saturday"})


### Categorical Variable Analysis

In [ ]:
bike.head()

In [ ]:
# Build boxplot of all categorical variables (before creating dummies) againt the target variable 'cnt' 
# to see how each of the predictor variable stackup against the target variable.

plt.figure(figsize=(20, 15))
plt.subplot(3,2,1)
sns.boxplot(x = 'season', y = 'cnt', data = bike)
plt.subplot(3,2,2)
sns.boxplot(x = 'weathersit', y = 'cnt', data = bike)
plt.subplot(3,2,3)
sns.boxplot(x = 'weekday', y = 'cnt', data = bike)
plt.subplot(3,2,4)
sns.boxplot(x = 'holiday', y = 'cnt', data = bike)
plt.subplot(3,2,5)
sns.boxplot(x = 'workingday', y = 'cnt', data = bike)
plt.subplot(3,2,6)
sns.boxplot(x = 'yr', y = 'cnt', data = bike)
plt.show()

From the above box plot we can infer the following:
- Season: Fall has the highest 'cnt' whereas Spring has the least 'cnt'
- Waethersit : Clear weather has the highest 'cnt' whereas 'Light_RainSnow' has the least 'cnt'
- Weekday: There is no particular day having significantly high 'cnt'. Almost all has the same 'cnt'
- holiday: There is high 'cnt' for days when there is no holiday.
- workingday: There is not significant difference in the working day and non-working day on the 'cnt'
- yr: 2019 has significantly high 'cnt' compared to '2018'

In [ ]:
# Created fucntion to analyse statistics of categorical features
def cat_stats(col):
    cat_df = bike.groupby(col)['cnt'].agg(['sum', 'mean','median','count']).sort_values('sum',ascending = False)
    cat_df['sum_perc']=cat_df['sum']/bike.cnt.sum()*100
    cat_df['count_perc']=cat_df['count']/bike.cnt.count()*100
    return round(cat_df,2)

In [ ]:
# Created function to generate plots for categorical features
def cat_plot(col,x,y):
    plt.figure(figsize = (x,y))
    plt.subplot(1,2,1)
    sns.barplot(col,'cnt',data=bike)
    plt.subplot(1,2,2)
    sns.barplot(col,'cnt',data=bike, hue='yr',palette='Paired')
    return

#### Season

In [ ]:
cat_stats('season')

In [ ]:
cat_plot('season',12,6)

Around 32% of the booking were happening in Fall with a median of over 5000 bookings. It is followed by Summer & Winter with 27% & 25% of total booking. 
It shows that season could be a good predictor of the dependent feature.

#### Weathersit

In [ ]:
cat_stats('weathersit')

In [ ]:
cat_plot('weathersit',18,6)

Around 68% of the bike booking was happening during Clear weather with a median 4844 bookings followed by Mist with 30% of the total booking. Weathersit shows some trend in the bike bookings, and could be a good predictor for the dependent variable. Also, the dataframe does not have any data where the weather is Heavy_RainSnow

#### Weekday

In [ ]:
cat_stats('weekday')

In [ ]:
cat_plot('weekday',18,6)

weekday variable shows close trend (between 13.5%-14.8% of total booking on all days of the week) having their independent medians between 4000 to 5000 bookings. This variable can have some or no influence on the predictor. Further analysis would be needed to determine whether this attribute needs to be included in the model parameter selection

#### holiday

In [ ]:
cat_stats('holiday')

In [ ]:
cat_plot('holiday',18,6)

97% of bike rentals is happening during non-holiday time.

#### Working Day

In [ ]:
cat_stats('workingday')

In [ ]:
cat_plot('workingday',18,6)

It can observed that there is almost same bookings in working day as well as non-working day. So, working day is not significant. But we can analyze it further during modelling.

#### Year

In [ ]:
cat_stats('yr')

In [ ]:
sns.barplot('yr','cnt',data=bike)
plt.show()

There is significant rise in the demand of boom bikes from 2018 to 2019. So, yr is highly significant.

#### Numerical Variable Analysis

In [ ]:
bike.head()

In [ ]:
#Generating heatmap to check the relationships between numeric variables variables
bike_num = bike[['temp_tf','atemp_tf','hum_tf','windspeed_tf','cnt']]
sns.heatmap(bike_num.corr(),annot=True)
#sns.pairplot(bike_num)
plt.show()

There is linear relationship between temp and atemp. Correlation coeff for temp_tf and atemp_tf is 0.99. Both of the parameters cannot be used in the model due to multicolinearity. Since atemp_tf is derived column we will drop it and keep the original temp_tf.

In [ ]:
bike.drop(["atemp_tf"],axis=1,inplace=True)

In [ ]:
#Generating pairplot to check the relationships between numeric variables variables
bike_num = bike[['temp_tf','hum_tf','windspeed_tf']]
sns.pairplot(bike_num)
plt.show()

So, we can observe from the pairplot that there is no significant correlation between the independent features

In [ ]:
# Checking the impact of year against the numerical variable : 
ax = sns.pairplot(x_vars=['temp_tf', 'hum_tf','windspeed_tf'], y_vars=['cnt'] , data=bike, hue='yr', palette='Set2')
ax._legend.remove()
plt.legend(labels=['2018', '2019'],loc=1)
plt.show()

As we can observe that there is linear relationship between temp, windspeed and cnt. There is a pattern observed with change in year.

### Data Preparation

#### Dummy Variable Creation

In [ ]:
# Let's drop the first column from season using 'drop_first = True'
season = pd.get_dummies(bike['season'],drop_first = True)

# Let's drop the first column from weekday using 'drop_first = True'
weekday = pd.get_dummies(bike['weekday'], drop_first = True)

# Let's drop the first column from weathersit using 'drop_first = True'
weathersit = pd.get_dummies(bike['weathersit'], drop_first = True)


#### Merging the Dataframes

In [ ]:
# Add the results to the original bike dataframe
df = pd.concat([bike, season,weathersit,weekday], axis = 1)

#### Removing unnecessary columns

In [ ]:
#deleting the unnecessry column season, weathersit and weekday as the respective values are already populated as binary columns data
df.drop(["season","weekday","weathersit"],axis=1,inplace=True)

In [ ]:
df.shape

In [ ]:
df.info()

All the 18 columns are numeric value. So, we can split the dataframe into Train & Test

### Splitting the data

In [ ]:
from sklearn.model_selection import train_test_split

# We specify this so that the train and test data set always have the same rows, respectively
np.random.seed(0)
df_train, df_test = train_test_split(df, train_size = 0.7, test_size = 0.3, random_state = 100)

#### Train Dataset

In [ ]:
df_train.shape

In [ ]:
df_train.describe()

#### Test Dataset

In [ ]:
df_test.shape

In [ ]:
df_test.describe()

Based on the 70% - 30% split between train and test dataset we have 510 rows in train dataset and 219 in test dataset

### Correlation Coeff

In [ ]:
plt.figure(figsize = (25,20))
ax= sns.heatmap(df_train.corr(), annot = True, cmap="RdYlGn",linewidth =1)
plt.show()

There is multi-colinearity between the variables. We can use vif in order to eliminate highly correlated features

- workingday variable has high negative correlation with Sat & Sun (where workingday =0)
- Spring is negatively correlated with cnt
- temp_tf and Spring has strong correlation
- mist weather and humidity has correlation

### Building the Linear Model

#### Dividing into X_train and y_train

In [ ]:
y_train = df_train['cnt']
X_train = df_train.drop(["cnt"],axis=1)

In [ ]:
X_train.shape

### RFE

In [ ]:
# Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [ ]:
# Running RFE with the output number of the variable equal to 15
lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, 15)             # running RFE
rfe = rfe.fit(X_train, y_train)

In [ ]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
#these are the columns we get after applying RFE
col = X_train.columns[rfe.support_]
col

#### VIF

In [ ]:
# Import VIF for checking the VIF values of the feature variables. 
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# Function for VIF Calculation

def calculateVIF(df):
    vif = pd.DataFrame()
    vif['Features'] = df.columns
    vif['VIF'] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return vif 

### Building model using statsmodel, for the detailed statistics

### Model 1

In [ ]:
# Creating X_test dataframe with RFE selected variables
X_train = X_train[col]

In [ ]:
# Adding a constant variable 
import statsmodels.api as sm  
X_train_rfe = sm.add_constant(X_train)

In [ ]:
lm1 = sm.OLS(y_train,X_train_rfe).fit()   # Running the linear model

In [ ]:
print(lm1.summary())

In [ ]:
calculateVIF(X_train)

Since Sunday has high p-value of 0.395 means it is statistically insignificant. We will go ahead with dropping Sunday from the model

In [ ]:
X_train = X_train.drop(["Sunday"], axis = 1)
X_train_rfe = sm.add_constant(X_train)

### Model 2

In [ ]:
lm2 = sm.OLS(y_train,X_train_rfe).fit()   # Running the linear model
print(lm2.summary())

In [ ]:
calculateVIF(X_train)

Since Saturday has high p-value of 0.706 means it is statistically insignificant. We will go ahead with dropping Saturday from the equation

In [ ]:
X_train = X_train.drop(["Saturday"], axis = 1)
X_train_rfe = sm.add_constant(X_train)

### Model 3

In [ ]:
lm3 = sm.OLS(y_train,X_train_rfe).fit()   # Running the linear model
print(lm3.summary())

In [ ]:
calculateVIF(X_train)

Since, workingday has high p-value of 0.362, p-value of greater than 0.05 is statistically insignificant. So we are dropping workingday from our model.

In [ ]:
X_train = X_train.drop(["workingday"], axis = 1)
X_train_rfe = sm.add_constant(X_train)

### Model 4

In [ ]:
lm4 = sm.OLS(y_train,X_train_rfe).fit()   # Running the linear model
print(lm4.summary())

In [ ]:
calculateVIF(X_train)

Now, we can observe none of the features in the model are multicollinear as vif<5

Although the Monday is statistically significant as per the p-value. But we will try to drop it to see whether the model improves or not

In [ ]:
X_train = X_train.drop(["Monday"], axis = 1)
X_train_rfe = sm.add_constant(X_train)

### Model 5

In [ ]:
lm5 = sm.OLS(y_train,X_train_rfe).fit()   # Running the linear model
print(lm5.summary())

As, we can observe from our model that Prob (F-statistic) improved from 6.55e-184 to 9.39e-184

In [ ]:
calculateVIF(X_train)

Although the Tuesday is statistically significant as per the p-value. But we will try to drop it to see whether the model improves or not

In [ ]:
X_train = X_train.drop(["Tuesday"], axis = 1)
X_train_rfe = sm.add_constant(X_train)

### Model 6

In [ ]:
lm6 = sm.OLS(y_train,X_train_rfe).fit()   # Running the linear model
print(lm6.summary())

As, we can observe from our model that F-statistic improved from 221.4 to 241.3

In [ ]:
calculateVIF(X_train)

Although the Summer is statistically significant as per the p-value. But we will try to drop it for better interpretation and will observe whether the model improves or not

In [ ]:
X_train = X_train.drop(["Summer"], axis = 1)
X_train_rfe = sm.add_constant(X_train)

### Model 7

In [ ]:
lm7 = sm.OLS(y_train,X_train_rfe).fit()   # Running the linear model
print(lm7.summary())

As, we can observe from our model that F-statistic improved from 241.3 to 263.3

In [ ]:
calculateVIF(X_train)

Since we are getting R-squared: 0.826 and Prob (F-statistic): 2.14e-183 and all the features are stastiscally significant as per the p-value and none are multicollinear as per the vif(under 2)

Model 7 is pretty good, as there is VERY LOW Multicollinearity between the predictors and the p-values for all the predictors seems to be significant. For now, we will consider this as our final model (unless the Test data metrics are not significantly close to this number)

In [ ]:
X_train_rfe.shape

### Hypothesis Testing :
Hypothesis Testing States that

H0:B1=B2=...=Bn=0

H1: at least one Bi!=0

where Bi are the coefficients

In [ ]:
# Checking the parameters obtained
lm7.params

From the lm7 model summary, it is evident that all coefficients are not equal to zero, which means we REJECT the NULL HYPOTHESIS

### F-Staitsics :
F-Statistics is used for testing the overall significance of the Model. The higher the F-Statistics, the more significant will be the Model.

F-Statistics : 263.3
    
Prob (F-statistic): 2.14e-183

## Residual Analysis of the train data

In [ ]:
y_train_pred = lm7.predict(X_train_rfe)

In [ ]:
# Plot the histogram of the error terms
fig = plt.figure()
sns.distplot((y_train - y_train_pred), bins = 20)
fig.suptitle('Error Terms', fontsize = 20)                  # Plot heading 
plt.xlabel('Errors', fontsize = 18)                         # X-label

In [ ]:
#mean of residuals
residuals= y_train -y_train_pred
m=np.mean(residuals)
m

In [ ]:
#r square value of model
r2_score(y_train,y_train_pred)

In [ ]:
#probability plot of predicted value
stats.probplot(y_train_pred, dist="norm", plot=py)
py.show()

### Making Predictions using final model

In [ ]:
df_test.head()

#### Dividing X_test and y_test

In [ ]:
y_test = df_test.pop('cnt')
X_test = df_test

In [ ]:
# Using model 7 to make predictions.

# Creating X_test_new dataframe by dropping variables from X_test
X_test_new = X_test[X_train.columns]

# Adding a constant variable 
X_test_new = sm.add_constant(X_test_new)

In [ ]:
# Making predictions using Model 7
y_pred = lm7.predict(X_test_new)

In [ ]:
# Plotting y_test and y_pred to observe the spread.
fig = plt.figure()
plt.scatter(y_test,y_pred)
fig.suptitle('y_test vs y_pred', fontsize=20)              # Plot heading 
plt.xlabel('y_test', fontsize=18)                          # X-label
plt.ylabel('y_pred', fontsize=16)                          # Y-label
plt.show()

In [ ]:
residual = y_test - y_pred

In [ ]:
#probability plot for predicted value
stats.probplot(y_pred, dist="norm", plot=py)
py.show()

In [ ]:
#Residual probability plot
fig, ax = plt.subplots(figsize=(6,2.5))
_, (__, ___, r) = stats.probplot(residual, plot=ax, fit=True)

#### Calculating R2 Value

In [ ]:
r2_score(y_test,y_pred)

#### Calculating RMSE for the selected Model

In [ ]:
RMSE = round(sqrt(mean_squared_error(y_test, y_pred)),4)
RMSE

#### Calculating Mean Absolute Error for the selected Model

In [ ]:
MAE = round(mean_absolute_error(y_test, y_pred),4)
MAE

Since, the R square value on the test dataset is 0.808 based on final model(Model 7) indicates that the model is really good.

## Assumptions of Linear Regression Model

In [ ]:
sm.graphics.plot_ccpr(lm7, 'temp_tf')
plt.show()

In [ ]:
sm.graphics.plot_ccpr(lm7, 'windspeed_tf')
plt.show()

The above plots represents the relationship between the model and the predictor variables. As we can see, linearity is  preserved

## No Multicolinearity

In [ ]:
vif = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
pd.DataFrame({'vif': vif[0:]}, index=X_train.columns).T

In [ ]:
calculateVIF(X_train)

All the predictor variables have VIF value less than 5. So we can consider that there is insignificant multicolinearity among the predictor variables.

# Homoscedasticity

In [ ]:
fig, ax = plt.subplots(figsize=(6,2.5))
_ = ax.scatter(y_pred, y_test-y_pred)

There is no visible pattern in residual values, thus homoscedacity is well preserved

## Checking autocorrelation of residuals

Autocorrelation refers to the fact that observations’ errors are correlated.

In [ ]:
acf = statsmodels.graphics.tsaplots.plot_acf(y_test - y_pred, lags=40 , alpha=0.05)
acf.show()

There is almost no autocorrelation.

## Model Outcome Summary

The equation of best fitted surface based on model lm7:

**cnt = 3864.992672 + (2007.219125 x yr) - ( 772.159535 x holiday) + ( 897.250477 x temp_tf) − (242.799026 x windspeed_tf) -(169.045308 x hum_tf) + (464.872957 x Winter) − (466.469063 x Mist) - (982.908490 X Spring)− (2134.477790 x Light_RainSnow)**

- yr : A coefficient value of ‘2007.219125’ indicated that a unit increase in yr variable, increases the bike hire numbers by 2007.219125 units


- temp_tf : A coefficient value of ‘897.250477’ indicated that a unit increase in temp variable, increases the bike hire numbers by 897.250477 units


- windspeed_tf : A coefficient value of ‘-242.799026’ indicated that, a unit increase in windspeed variable decreases the bike hire numbers by 242.799026 units


- Winter : A coefficient value of ‘464.872957’ indicated that a unit increase in Winter variable increases the bike hire numbers by 464.872957 units


- light_RainSnow : A coefficient value of ‘-2134.477790’ indicated that, a unit increase in Light_RainSnow variable, decreases the bike hire numbers by -2134.477790 units


- Mist : A coefficient value of ‘-466.469063’ indicated that a unit increase in Mist weather variable, decreases the bike hire numbers by 466.469063 units


- holiday : A coefficient value of ‘-772.159535’ indicated that a unit increase in holiday variable decreases the bike hire numbers by 772.159535 units


- Spring: A coefficient value of ‘-982.908490’ indicated that a unit increase in Spring variable decreases the bike hire numbers by 982.908490 units


- hum_tf : A coefficient value of ‘-169.045308’ indicated that a unit increase in hum_tf variable decreases the bike hire numbers by 169.045308 units

### As per the final model, the top 3 predictor variables that influences bike booking are:

- Year (yr)
A coefficient value of ‘2007.219125’ indicated that a unit increase in yr variable, increases the bike hire numbers by 2007.219125 units

- Temperature (temp_tf)
A coefficient value of ‘897.250477’ indicated that a unit increase in temp variable, increases the bike hire numbers by 897.250477 units

- Light Rain & Snow (weathersit =3)
A coefficient value of ‘-2134.477790’ indicated that, a unit increase in Light_RainSnow variable, decreases the bike hire numbers by -2134.477790 units

It is recommended to give importance to these three variables while planning to achieve maximum bike rental booking.
As high temperature and good weather positively impacts bike rentals, it is recommended that bike availability and promotions to be increased during these months to further increase bike rentals.